In [26]:
import pandas as pd
import geopandas as gp
import numpy as np
import os

In [28]:
rdh_list = os.listdir("./al_rdh_processed_csvs/zipped_files/") 
ochoa_list = os.listdir("./s_ochoa_data/")

In [29]:
print(rdh_list)

['al_pl2020_sldl', 'al_pl2020_t', 'al_pl2020_cd', 'al_pl2020_sldu', 'al_pl2020_vtd', 'al_pl2020_bg', 'al_pl2020_st', 'al_pl2020_cnty', 'al_pl2020_b', 'al_pl2020_place']


In [30]:
print(ochoa_list)

['County_al_legacy_formatted_short.csv', 'State_al_legacy_formatted_short.csv', 'Place_al_legacy_formatted_short.csv', 'StateLower_al_legacy_formatted_short.csv', 'StateUpper_al_legacy_formatted_short.csv', 'Congress_al_legacy_formatted_short.csv']


In [31]:
ochoa_files = {}
for file in ochoa_list:
    file_string = "./s_ochoa_data/"+file
    file_actual = pd.read_csv(file_string)
    ochoa_files[file]=file_actual

In [32]:
#This is where I got these names from (~page 99): 
#https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/complete-tech-docs/summary-file/2020Census_PL94_171Redistricting_StatesTechDoc_English.pdf

col_adjustment_dict = {
'PL2020_Population':'P0010001',
    'PL2020_Hispanic_Origin':'P0020002', 
    'PL2020_Not_Hispanic':'P0020003',
    'PL2020_NH_1_Race':'P0020004',
    'PL2020_NH_Wht':"P0020005", 
    'PL2020_NH_Blk':"P0020006", 
    'PL2020_NH_Ind':"P0020007", 
    'PL2020_NH_Asn':"P0020008",
    'PL2020_NH_Hwn':"P0020009",
    'PL2020_NH_Oth':"P0020010",
    'PL2020_NH_2plus_Races':"P0020011",
    'PL2020_NH_3_Races':"P0020028", 
    'PL2020_NH_4_Races':"P0020049", 
    'PL2020_NH_5_Races':"P0020065",
    'PL2020_NH_6_Races':"P0020072",
    'PL2020_18_Pop':"P0040001", 
    'PL2020_H18_Pop':"P0040002",
    'PL2020_NH18_Pop':"P0040003", 
    'PL2020_NH18_1_Race':"P0040004",
    'PL2020_NH18_Wht':"P0040005",
    'PL2020_NH18_Blk':"P0040006",
    'PL2020_NH18_Ind':"P0040007",
    'PL2020_NH18_Asn':"P0040008",
    'PL2020_NH18_Hwn':"P0040009", 
    'PL2020_NH18_Oth':"P0040010", 
    'PL2020_NH18_2plus_Races':"P0040011",
    'PL2020_NH18_2_Races':"P0040012", 
    'PL2020_NH18_3_Races':"P0040028", 
    'PL2020_NH18_4_Races':"P0040049",
    'PL2020_NH18_5_Races':"P0040065",
    'PL2020_NH18_6_Races':"P0040072",
    'PL2020_HH':"H0010001",
    'PL2020_HH_Occupied':"H0010002",
    'PL2020_HH_Vacant':"H0010003",
    'PL2020_People_in_group_quarters':"P0050001", 
    'PL2020_InGrp_Institutionalized':"P0050002",
    'PL2020_InGrp_Adult_Correctional':"P0050003", 
    'PL2020_InGrp_Juvenile':"P0050004",
    'PL2020_InGrp_Nursing':"P0050005", 
    'PL2020_InGrp_Other_Institutionalized':"P0050006",
    'PL2020_InGrp_Noninstitutionalized':"P0050007", 
    'PL2020_InGrp_College_Student':"P0050008",
    'PL2020_InGrp_Military':"P0050009", 
    'PL2020_InGrp_Other_Noninstitutionalized':"P0050010"}

In [33]:
for val in ochoa_files:
    ochoa_files[val] = ochoa_files[val].rename(columns=col_adjustment_dict)


In [34]:
rdh_files = {}
for file in rdh_list:
    #print(file)
    if file != ".DS_Store":
        file_string = "./al_rdh_processed_csvs/zipped_files/"+file+"/"+file+".csv"
        file_actual = pd.read_csv(file_string)
        rdh_files[file]=file_actual

In [35]:
def census_counts_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(O)","{:.>5}".format(int(row[left_data]))," (RDH){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(RDH)","{:.>5}".format(int(row[left_data]))," (O){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "rows with a difference greater than 10")
    print("")
    print("All rows containing differences:")
    diff_list.sort()
    print(diff_list)

In [36]:
column_list = ['P0010001', 'P0020002', 'P0020003',
       'P0020004', 'P0020005', 'P0020006', 'P0020007', 'P0020008', 'P0020009',
       'P0020010', 'P0020011', 'P0020028', 'P0020049', 'P0020065', 'P0020072',
       'P0040001', 'P0040002', 'P0040003', 'P0040004', 'P0040005', 'P0040006',
       'P0040007', 'P0040008', 'P0040009', 'P0040010', 'P0040011', 'P0040012',
       'P0040028', 'P0040049', 'P0040065', 'P0040072', 'H0010001', 'H0010002',
       'H0010003', 'P0050001', 'P0050002', 'P0050003', 'P0050004', 'P0050005',
       'P0050006', 'P0050007', 'P0050008', 'P0050009', 'P0050010']

In [37]:
to_compare = {'County_al_legacy_formatted_short.csv':'al_pl2020_cnty',
             'State_al_legacy_formatted_short.csv':'al_pl2020_st',
             'Place_al_legacy_formatted_short.csv':'al_pl2020_place',
             'StateLower_al_legacy_formatted_short.csv':'al_pl2020_sldl',
             'StateUpper_al_legacy_formatted_short.csv':'al_pl2020_sldu',
             'Congress_al_legacy_formatted_short.csv':'al_pl2020_cd'}

for val in to_compare:
    print(val)
    temp_join = pd.merge(ochoa_files[val],rdh_files[to_compare[val]],how="inner",on="GEOID",indicator=True)
    print(temp_join["_merge"].value_counts())
    census_counts_check(temp_join,column_list,True,"GEOID",print_level=0)
    print()
    print("****************************")

County_al_legacy_formatted_short.csv
both          67
left_only      0
right_only     0
Name: _merge, dtype: int64

There are  67  total rows
0  of these rows have differences
67  of these rows are the same

The max difference between any one shared column in a row is:  0
There are  0 rows with a difference greater than 10

All rows containing differences:
[]

****************************
State_al_legacy_formatted_short.csv
both          1
left_only     0
right_only    0
Name: _merge, dtype: int64

There are  1  total rows
0  of these rows have differences
1  of these rows are the same

The max difference between any one shared column in a row is:  0
There are  0 rows with a difference greater than 10

All rows containing differences:
[]

****************************
Place_al_legacy_formatted_short.csv
both          593
left_only       0
right_only      0
Name: _merge, dtype: int64

There are  593  total rows
0  of these rows have differences
593  of these rows are the same

The max di